<a href="https://colab.research.google.com/github/iragnis/Drone-Image-Position/blob/main/final_geo_drone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster,MousePosition
import re

In [ ]:
def data(df):
  ''' Cleaning and transforming the dataframe into the format required for mapping'''
  #inserting a null row on top
  # df1 = pd.DataFrame([[np.nan] * len(df.columns)], columns=df.columns)
  # df = df1.append(df, ignore_index=True)
  #copying column values to first row
  # df.iloc[0]=df.columns[0]
  #splitting the single column into Latitude and Longitude columns
  df[['Latitude','Longitude']] = df[df.columns[0]].str.split(',',expand=True)
  #dropping the first column
  df = df.iloc[: , 1:]
  #extracting values by using , as delimiter by converting into str
  df['Latitude']=df['Latitude'].astype('str').str.split(':').str[1]
  df['Longitude']=df['Longitude'].astype('str').str.split(':').str[1]
  #converting values into numeric data type for visual mapping
  df['Latitude']=pd.to_numeric(df['Latitude'],errors='coerce')
  df['Longitude']=pd.to_numeric(df['Longitude'],errors='coerce')
  return df

In [ ]:
def csv_conv(file):
  '''converting .srt file into a dataframe by extracting the latitude and longitude with 
  respect to the timestamp at an interval of 900 seconds'''
      with open(file, 'r') as h:
          sub = h.readlines()
      re_pattern = r'[0-9]{2}:[0-9]{2}:[0-9]{2},[0-9]{3} -->'
      regex = re.compile(re_pattern)
       # Get lines
      lines = [[]]
      for sentence in sub:
          if re.match(re_pattern, sentence):
              lines[-1].pop()
              lines.append([])
            
          else:
              lines[-1].append(sentence)
              
      lines = lines[1:]         
      l=[]
      for idx,row in lines:
            l.append(idx)
     
      df=pd.DataFrame(l)
      df=data(df)
      return df

In [ ]:
def marker(df):
  ''' Mapping multiple points of the dataframe '''
       m_2_f= folium.Map(location=[df['Latitude'].mean(),df['Longitude'].mean()], tiles='cartodbpositron', zoom_start=14.7)
       # Add points to the map
       for idx, row in df.iterrows():
         Marker([row['Latitude'], row['Longitude']]).add_to(m_2_f)
      # Display the map
       formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

       MousePosition(
            position="topright",
            separator=" | ",
            empty_string="NaN",
            lng_first=True,
            num_digits=20,
            prefix="Coordinates:",
            lat_formatter=formatter,
            lng_formatter=formatter,
        ).add_to(m_2_f)
       return m_2_f

In [ ]:
def heatmap(df):
      # Create a base map
      m_5 = folium.Map(location=[df['Latitude'].mean(),df['Longitude'].mean()], tiles='cartodbpositron', zoom_start=15)
      # Add a heatmap to the base map
      HeatMap(data=df[['Latitude', 'Longitude']], radius=10).add_to(m_5)
      # Display the map
      return m_5

In [ ]:
df=pd.DataFrame()
df=csv_conv("/content/1.srt")
df.head()


,Latitude,Longitude
0,12.942753,77.534032
1,12.942752,77.534032
2,12.942753,77.534032
3,12.942752,77.534031
4,12.942753,77.534031


In [ ]:
df2=pd.DataFrame()
df2=csv_conv("/content/DJI_0001.srt")
df2.head()


,Latitude,Longitude
0,12.809886,77.415549
1,12.809886,77.415549
2,12.809886,77.415549
3,12.809886,77.415549
4,12.809886,77.415549


In [ ]:
marker(df2)

In [ ]:
heatmap(df2)

In [ ]:
marker(df)

In [ ]:
heatmap(df)